In [1]:
import gradio as gr
import datetime
from PIL import Image
import tempfile
import cv2
import os
print("📁 Current Working Directory:", os.getcwd())
print("📂 Contents of ./models:", os.listdir("models"))


# Import your evaluation function
from pipeline.evaluator import evaluate_image

# === CONFIG ===
MODEL_WEIGHTS_PATH = r"models\densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
STARDIST_MODEL_DIR = os.path.join("stardist_model")

# === ANALYSIS FUNCTION ===
def analyze_slide(patient_id, slide_id, image):
    # Save uploaded image temporarily
    with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp:
        image.save(temp.name)
        temp_path = temp.name

    # Run the pipeline
    overlay_img, results = evaluate_image(temp_path, MODEL_WEIGHTS_PATH, STARDIST_MODEL_DIR)

    # Clean up input file (optional)
    os.remove(temp_path)

    if results is None:
        prediction = "Normal"
        alert_msg = "✅ No abnormality detected."
        cell_distribution = {
            "Normal": 1, "Low Grade": 0, "High Grade": 0, "Cancer": 0
        }
    else:
        prediction = results["majority_class"].replace("_", " ").capitalize()
        alert_msg = results["alert_message"]
        cell_distribution = results["class_counts"]

    # Current date
    date_today = datetime.datetime.now().strftime("%d %b %Y")

    # Summary text
    llm_summary = f"""Analysis Summary:
- Cervical LBC smear indicates {prediction.lower()} features.
- Immediate review recommended.""" if prediction != "Normal" else \
    "Analysis Summary:\n- No abnormal cells detected.\n- Routine follow-up advised."

    # Cell-wise breakdown
    classwise_output = (
        f"Normal Cells: {cell_distribution.get('normal', 0)}\n"
        f"Low Grade Cells: {cell_distribution.get('low_grade', 0)}\n"
        f"High Grade Cells: {cell_distribution.get('high_grade', 0)}\n"
        f"Cancerous Cells: {cell_distribution.get('cancer', 0)}"
    )

    # Convert overlayed image to PIL for display
    overlay_pil = Image.fromarray(cv2.cvtColor(overlay_img, cv2.COLOR_BGR2RGB))

    return (
        date_today,
        patient_id,
        slide_id,
        prediction,
        overlay_pil,
        alert_msg,
        classwise_output,
        llm_summary
    )

# === GRADIO UI ===
with gr.Blocks(css="styles.css") as demo:
    gr.Markdown("# 🧪 Cervical Cytology AI Analysis Tool")
 
    with gr.Row():
        # LEFT PANEL
        with gr.Column(scale=1):
            gr.Markdown("### 📤 Upload Cervical LBC Slide and Patient Details")

            with gr.Row():
                patient_id = gr.Textbox(label="Patient ID", placeholder="e.g. P01033")
                slide_id = gr.Textbox(label="Slide ID", placeholder="e.g. SL1706")

            image_input = gr.Image(label="Upload Slide Image (LBC only)", type="pil")

            analyze_btn = gr.Button("🔍 Analyze Slide")
            alert_output = gr.Textbox(label="Alert Message", lines=1, interactive=False)

            llm_output = gr.Textbox(label="LLM-generated Analysis", lines=6, interactive=False)

        # RIGHT PANEL
        with gr.Column(scale=1):
            date_output = gr.Textbox(label="Date", interactive=False)

            with gr.Row():
                patient_id_display = gr.Textbox(label="Patient ID", interactive=False)
                slide_id_display = gr.Textbox(label="Slide ID", interactive=False)

            prediction_output = gr.Textbox(label="Prediction", interactive=False, elem_id="prediction-box")

            image_output = gr.Image(label="Overlayed Cytology Image")

            cell_dist_output = gr.Textbox(label="Cell Type Distribution", lines=6, interactive=False)

    # Button event binding
    analyze_btn.click(
        fn=analyze_slide,
        inputs=[patient_id, slide_id, image_input],
        outputs=[
            date_output,
            patient_id_display,
            slide_id_display,
            prediction_output,
            image_output,
            alert_output,
            cell_dist_output,
            llm_output
        ]
    )

# Run the Gradio app
demo.launch()


📁 Current Working Directory: c:\Users\HFX1KOR\Desktop\gradio\GradioXAI
📂 Contents of ./models: ['densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5', 'densenet_checkpoint.weights.h5']


__init__.py (36): h5py is running against HDF5 1.14.5 when it was built against 1.14.6, this may cause problems


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\HFX1KOR\.conda\envs\cervical_env\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\HFX1KOR\.conda\envs\cervical_env\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\HFX1KOR\.conda\envs\cervical_env\lib\site-packages\gradio\blocks.py", line 2220, in process_api
    result = await self.call_function(
  File "c:\Users\HFX1KOR\.conda\envs\cervical_env\lib\site-packages\gradio\blocks.py", line 1731, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "c:\Users\HFX1KOR\.conda\envs\cervical_env\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "c:\Users\HFX1KOR\.conda\envs\cervical_env\lib\site-packages\anyio\_backends\_asyncio.py", line 2505, in run_sync